#### Sobre o Conjunto de Dados
Este é um conjunto de dados público de pedidos do e-commerce brasileiro feitos na loja Olist. O conjunto de dados contém informações de 100 mil pedidos de 2016 a 2018 feitos em diversos marketplaces no Brasil.
Esses são dados comerciais reais, que foram anonimizados, e as referências às empresas e parceiros nos textos de avaliação foram substituídas pelos nomes das grandes casas de Game of Thrones.


#### Contexto
Este conjunto de dados foi generosamente fornecido pela Olist, a maior loja de departamentos dos marketplaces brasileiros. A Olist conecta pequenos negócios de todo o Brasil a canais de vendas, sem complicações e com um único contrato. Esses comerciantes conseguem vender seus produtos através da Olist Store e enviá-los diretamente aos clientes utilizando os parceiros logísticos da Olist. 

Após o cliente comprar um produto na Olist Store, o vendedor é notificado para atender a esse pedido. Uma vez que o cliente recebe o produto, ou que a data estimada de entrega expire, o cliente recebe uma pesquisa de satisfação por e-mail, onde pode dar uma nota para a experiência de compra e escrever alguns comentários.



<img src='https://i.imgur.com/HRhd2Y0.png' height=400px/>

In [44]:
import polars as pl
import pandas as pd

#### Informações adcionais
----------

##### Conjunto de Dados de Clientes(olist_customer)
> Este conjunto de dados contém informações sobre o cliente e sua localização. Utilize-o para identificar clientes únicos no conjunto de dados de pedidos e para encontrar a localização de entrega dos pedidos.
>
> Em nosso sistema, cada pedido é atribuído a um customer_id exclusivo. Isso significa que o mesmo cliente terá diferentes ids para pedidos distintos. O propósito de ter um customer_unique_id no conjunto de dados é permitir que você identifique clientes que fizeram novas compras na loja. Caso contrário, você descobriria que cada pedido estaria associado a um cliente diferente.
--------

##### Conjunto de Dados de Geolocalização (geolocation)
> Este conjunto de dados contém informações sobre os códigos postais brasileiros e suas coordenadas de latitude/longitude. Utilize-o para plotar mapas e calcular distâncias entre vendedores e clientes

--------

##### Conjunto de Dados de Itens de Pedidos (order_items)
> Este conjunto de dados inclui informações sobre os itens comprados em cada pedido.

> Exemplo:
> O order_id = 00143d0f86d6fbd9f9b38ab440ac16f5 possui 3 itens (mesmo produto). Cada item tem o frete calculado de acordo com suas medidas e peso. Para obter o valor total do frete de cada pedido, basta somar.


-------
##### Payments Dataset
> This dataset includes data about the orders payment options.

-------
##### Conjunto de Dados de Avaliações de Pedidos (reviews dataset) 
> Este conjunto de dados inclui informações sobre as avaliações feitas pelos clientes.
> Após o cliente comprar um produto na Olist Store, o vendedor é notificado para atender a esse pedido. Quando o cliente recebe o produto, ou quando a data estimada de entrega se expira, o cliente recebe uma pesquisa de satisfação por e-mail, onde pode dar uma nota para a experiência de compra e escrever alguns comentários.

&nbsp;

### Limpeza de dados

#### Limpeza para os Dados de Clientes(olist_customer)

In [45]:
customer = pl.read_csv(
    source='dataset/olist_customers_dataset.csv', infer_schema=False
)

In [46]:
customer.head(2)

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
str,str,str,str,str
"""06b8999e2fba1a1fbc88172c00ba8b…","""861eff4711a542e4b93843c6dd7feb…","""14409""","""franca""","""SP"""
"""18955e83d337fd6b2def6b18a428ac…","""290c77bc529b7ac935b93aa66c333d…","""09790""","""sao bernardo do campo""","""SP"""


In [47]:
customer.shape

(99441, 5)

Verificando a quantidade de valores únicos de cada coluna

In [48]:
pd.DataFrame({
    col : customer.select(pl.col(col)).n_unique() \
    for col in customer.columns
}, index=['Quantidade de valores únicos']).T

,Quantidade de valores únicos
customer_id,99441
customer_unique_id,96096
customer_zip_code_prefix,14994
customer_city,4119
customer_state,27


> `customer_id` é uma chave primaria
>
> `customer_zip_code_prefix` é uma chave estrangeira

Chave primaria e chave estrangeira não necessitam de tratamento, somente, se houver valores nulos nesse conj. dados. Mais abaixo irei verificar a quantidade de valores nulos por colunas

In [49]:
customer.select(pl.col('customer_id').null_count()).item()

0

In [50]:
customer\
    .null_count()\
    .transpose(
        include_header=True,
        header_name   ='colunas',
        column_names  =['Quantidade de valores nulos']
).to_pandas()

,colunas,Quantidade de valores nulos
0,customer_id,0
1,customer_unique_id,0
2,customer_zip_code_prefix,0
3,customer_city,0
4,customer_state,0


> Não há valores nulos nas colunas acima

fazendo uma analise nas colunas `customer_city` e `customer_state`

In [51]:
customer.select(pl.col('customer_city')).head(3)

customer_city
str
"""franca"""
"""sao bernardo do campo"""
"""sao paulo"""


> vou deixar no formato de titulo o texto para melhor visualização para futura analises

In [52]:
customer = customer.with_columns(
    pl.col('customer_city').cast(pl.String).str.to_titlecase()
)

In [53]:
customer.select(pl.col('customer_city')).head(10)

customer_city
str
"""Franca"""
"""Sao Bernardo Do Campo"""
"""Sao Paulo"""
"""Mogi Das Cruzes"""
"""Campinas"""
"""Jaragua Do Sul"""
"""Sao Paulo"""
"""Timoteo"""
"""Curitiba"""


In [54]:
# Escrevendo dados
customer.write_csv(file='dataset_limpo_para_analise/olist_customers.csv')

&nbsp;

#### Limpeza para os Dados de Clientes(olist_customer)

In [55]:
order = pl.read_csv('dataset/olist_orders_dataset.csv', infer_schema=False)
order.head(3)

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
str,str,str,str,str,str,str,str
"""e481f51cbdc54678b7cc49136f2d6a…","""9ef432eb6251297304e76186b10a92…","""delivered""","""2017-10-02 10:56:33""","""2017-10-02 11:07:15""","""2017-10-04 19:55:00""","""2017-10-10 21:25:13""","""2017-10-18 00:00:00"""
"""53cdb2fc8bc7dce0b6741e21502734…","""b0830fb4747a6c6d20dea0b8c802d7…","""delivered""","""2018-07-24 20:41:37""","""2018-07-26 03:24:27""","""2018-07-26 14:31:00""","""2018-08-07 15:27:45""","""2018-08-13 00:00:00"""
"""47770eb9100c2d0c44946d9cf07ec6…","""41ce2a54c0b03bf3443c3d931a3670…","""delivered""","""2018-08-08 08:38:49""","""2018-08-08 08:55:23""","""2018-08-08 13:50:00""","""2018-08-17 18:06:29""","""2018-09-04 00:00:00"""


In [56]:
order['order_status'].unique()

order_status
str
"""delivered"""
"""created"""
"""unavailable"""
"""shipped"""
"""processing"""
"""invoiced"""
"""approved"""
"""canceled"""


In [57]:
status_map = {
    "delivered": "entregue",
    "invoiced": "faturado",
    "shipped": "enviado",
    "created": "criado",
    "approved": "aprovado",
    "processing": "processando",
    'canceled' :'cancelado',
    'unavailable': 'indisponível'
}

In [58]:
order = order.with_columns(
    pl.col("order_status").replace(status_map)
)

In [59]:
order['order_status'].unique()

order_status
str
"""enviado"""
"""faturado"""
"""aprovado"""
"""indisponível"""
"""cancelado"""
"""entregue"""
"""processando"""
"""criado"""


- "enviado", 'entrege','aprovado, 'entregue', 'faturado' = 'ok'
- 'cancelado', 'indisponivel' = no
- 'processado'  = 'em processo'

In [60]:
ok = ['enviado', 'entrege','aprovado', 'entregue', 'faturado']
no = ['cancelado', 'indisponível']
in_ = ['processando']

order = order.with_columns(
    status_ = pl.when(pl.col('order_status').is_in(ok))\
                .then(pl.lit('ok'))\
                .when(pl.col('order_status').is_in(no))\
                .then(pl.lit('no'))
                .otherwise(pl.lit('in_'))
)

In [61]:
order\
    .null_count()\
    .transpose(
        include_header=True,
        header_name   ='colunas',
        column_names  =['Quantidade de valores nulos']
)

colunas,Quantidade de valores nulos
str,u32
"""order_id""",0
"""customer_id""",0
"""order_status""",0
"""order_purchase_timestamp""",0
"""order_approved_at""",160
"""order_delivered_carrier_date""",1783
"""order_delivered_customer_date""",2965
"""order_estimated_delivery_date""",0
"""status_""",0


> Os valores nan serão removidas

In [62]:
order.write_csv(
    'dataset_limpo_para_analise\\olist_order.csv'
)

#### limpeza para Itens de Pedidos (order_items)

In [63]:
pedidos = pl.read_csv(
    source='dataset/olist_order_items_dataset.csv',
    infer_schema=False
    )

In [64]:
pedidos.columns

['order_id',
 'order_item_id',
 'product_id',
 'seller_id',
 'shipping_limit_date',
 'price',
 'freight_value']

In [65]:
pedidos.head(3)

order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
str,str,str,str,str,str,str
"""00010242fe8c5a6d1ba2dd792cb162…","""1""","""4244733e06e7ecb4970a6e2683c13e…","""48436dade18ac8b2bce089ec2a0412…","""2017-09-19 09:45:35""","""58.90""","""13.29"""
"""00018f77f2f0320c557190d7a144bd…","""1""","""e5f2d52b802189ee658865ca93d83a…","""dd7ddc04e1b6c2c614352b383efe2d…","""2017-05-03 11:05:13""","""239.90""","""19.93"""
"""000229ec398224ef6ca0657da4fc70…","""1""","""c777355d18b72b67abbeef9df44fd0…","""5b51032eddd242adc84c38acab88f2…","""2018-01-18 14:48:30""","""199.00""","""17.87"""


In [66]:
pedidos\
    .select(pl.all().n_unique())\
    .transpose(
        include_header=True,
        header_name='colunas',
        column_names=['Valores Únicos']
    )

colunas,Valores Únicos
str,u32
"""order_id""",98666
"""order_item_id""",21
"""product_id""",32951
"""seller_id""",3095
"""shipping_limit_date""",93318
"""price""",5968
"""freight_value""",6999


In [67]:
pedidos.select(
    pl.all().null_count()
).transpose(
    include_header=True,
    header_name ='colunas',
    column_names=['Quantidade de valores nulos']
)

colunas,Quantidade de valores nulos
str,u32
"""order_id""",0
"""order_item_id""",0
"""product_id""",0
"""seller_id""",0
"""shipping_limit_date""",0
"""price""",0
"""freight_value""",0


> Não há valores nulos nesse conjunto de dados

Conventendo valores numericos que estão no formato string, e conventendo para o padrão brasileiro a data

In [68]:
pedidos = pedidos.with_columns(
        pl.col('shipping_limit_date')\
            .str.strptime(pl.Datetime(time_zone='UTC'))\
            .dt.strftime(format="%d-%m-%Y %H:%M:%S"), # Convertendo data

        pl.col('price').str.to_decimal(),         # Convertendo para float
        pl.col('freight_value').str.to_decimal(), # Convertendo para float
    )

In [69]:
pedidos.dtypes

[String,
 String,
 String,
 String,
 String,
 Decimal(precision=None, scale=2),
 Decimal(precision=None, scale=2)]

In [70]:
# Escrevendo dados
pedidos.write_csv(file='dataset_limpo_para_analise/olist_order_items.csv')

&nbsp;

#### limpeza Payments Dataset(olist_payments)

In [71]:
payments = pl.read_csv(
    'dataset/olist_order_payments_dataset.csv',
    infer_schema=False
)

In [72]:
payments.head(2)

order_id,payment_sequential,payment_type,payment_installments,payment_value
str,str,str,str,str
"""b81ef226f3fe1789b1e8b2acac839d…","""1""","""credit_card""","""8""","""99.33"""
"""a9810da82917af2d9aefd1278f1dcf…","""1""","""credit_card""","""1""","""24.39"""


In [73]:
payments.select(
    pl.all().null_count()
).transpose(
    include_header=True,
    header_name ='colunas',
    column_names=['Quantidade de valores nulos']
)

colunas,Quantidade de valores nulos
str,u32
"""order_id""",0
"""payment_sequential""",0
"""payment_type""",0
"""payment_installments""",0
"""payment_value""",0


In [74]:
payments\
    .select(pl.all().n_unique())\
    .transpose(
        include_header=True,
        header_name='colunas',
        column_names=['Valores Únicos']
    )

colunas,Valores Únicos
str,u32
"""order_id""",99440
"""payment_sequential""",29
"""payment_type""",5
"""payment_installments""",24
"""payment_value""",29077


In [75]:
payments\
    .select(pl.col('payment_type'))\
    .group_by(by=pl.col('payment_type'))\
    .len()\
    .sort(by=pl.col('len'))

by,len
str,u32
"""not_defined""",3
"""debit_card""",1529
"""voucher""",5775
"""boleto""",19784
"""credit_card""",76795


> Não definido será transformando para nada e removido

In [76]:
payments = payments\
    .with_columns(
        pl.when(pl.col('payment_type')=='not_defined')\
          .then(pl.lit(None))\
          .otherwise(pl.col('payment_type')).alias(name='payment_type')
    )

In [77]:
payments.select(
    pl.all().null_count()
).transpose(
    include_header=True,
    header_name ='colunas',
    column_names=['Quantidade de valores nulos']
)

colunas,Quantidade de valores nulos
str,u32
"""order_id""",0
"""payment_sequential""",0
"""payment_type""",3
"""payment_installments""",0
"""payment_value""",0


In [78]:
payments.head(3)

order_id,payment_sequential,payment_type,payment_installments,payment_value
str,str,str,str,str
"""b81ef226f3fe1789b1e8b2acac839d…","""1""","""credit_card""","""8""","""99.33"""
"""a9810da82917af2d9aefd1278f1dcf…","""1""","""credit_card""","""1""","""24.39"""
"""25e8ea4e93396b6fa0d3dd708e76c1…","""1""","""credit_card""","""1""","""65.71"""


In [79]:
payments = payments.with_columns(
    pl.when(pl.col('payment_type') == 'credit_card')\
        .then(pl.lit('Cartão de Crédito'))\
        .when(pl.col('payment_type') == 'boleto')\
        .then(pl.lit('Boleto'))\
        .when(pl.col('payment_type')=='debit_card')\
        .then(pl.lit('Cartão de debito'))\
        .otherwise(pl.col('payment_type'))\
        .alias('payment_type')
)

In [80]:
payments\
    .select(pl.col('payment_type'))\
    .group_by(by=pl.col('payment_type'))\
    .len()\
    .sort(by=pl.col('len'))

by,len
str,u32
null,3
"""Cartão de debito""",1529
"""voucher""",5775
"""Boleto""",19784
"""Cartão de Crédito""",76795


In [81]:
payments.to_pandas().dropna().to_csv('dataset_limpo_para_analise/olist_order_payments.csv', index=False)

&nbsp;

#### limpeza para Reviews(olist_Reviews)

In [82]:
reviews = pl.read_csv(
    source='dataset/olist_order_reviews_dataset.csv', infer_schema=False
)

In [83]:
reviews\
    .select(
        pl.all().null_count()
        )\
    .transpose(
        include_header=True,
        header_name ='colunas',
        column_names=['Quantidade de valores nulos']
    )

colunas,Quantidade de valores nulos
str,u32
"""review_id""",0
"""order_id""",0
"""review_score""",0
"""review_comment_title""",87656
"""review_comment_message""",58247
"""review_creation_date""",0
"""review_answer_timestamp""",0


In [84]:
reviews.columns

['review_id',
 'order_id',
 'review_score',
 'review_comment_title',
 'review_comment_message',
 'review_creation_date',
 'review_answer_timestamp']

In [85]:
reviews = reviews.with_columns(
    pl.col('review_comment_title').fill_null("Não colocou titulo")
)

In [86]:
reviews = reviews.with_columns(
    pl.col('review_comment_message').fill_null("Não comentou")
)

In [87]:
reviews\
    .select(
        pl.all().null_count()
        )\
    .transpose(
        include_header=True,
        header_name ='colunas',
        column_names=['Quantidade de valores nulos']
    )

colunas,Quantidade de valores nulos
str,u32
"""review_id""",0
"""order_id""",0
"""review_score""",0
"""review_comment_title""",0
"""review_comment_message""",0
"""review_creation_date""",0
"""review_answer_timestamp""",0


In [88]:
reviews.write_csv("dataset_limpo_para_analise/olist_order_review.csv")

&nbsp;

#### Limpeza para sellers(olist_sellers)

In [89]:
sellers = pl.read_csv(
    "dataset/olist_sellers_dataset.csv", infer_schema=False
)
sellers.head(2)

seller_id,seller_zip_code_prefix,seller_city,seller_state
str,str,str,str
"""3442f8959a84dea7ee197c632cb2df…","""13023""","""campinas""","""SP"""
"""d1b65fc7debc3361ea86b5f14c68d2…","""13844""","""mogi guacu""","""SP"""


In [90]:
sellers.select(
    pl.all().null_count()
).transpose(include_header=True, 
            header_name="Colunas", 
            column_names=["Quantidade de valores nulos"])

Colunas,Quantidade de valores nulos
str,u32
"""seller_id""",0
"""seller_zip_code_prefix""",0
"""seller_city""",0
"""seller_state""",0


> Não há valores nulos no conjunto de dos

In [91]:
# Modificando o nome das cidades para o tipo de titulo
sellers = sellers.with_columns(
    pl.col("seller_city").str.to_titlecase()
)
sellers.head(2)

seller_id,seller_zip_code_prefix,seller_city,seller_state
str,str,str,str
"""3442f8959a84dea7ee197c632cb2df…","""13023""","""Campinas""","""SP"""
"""d1b65fc7debc3361ea86b5f14c68d2…","""13844""","""Mogi Guacu""","""SP"""


In [92]:
sellers.write_csv(
    "dataset_limpo_para_analise/olist_sellers.csv"
)

&nbsp;

#### limpeza para products(olist_product)

In [93]:
product = pl.read_csv(
    source = "dataset/olist_products_dataset.csv"
)
product.head(2)

product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
str,str,i64,i64,i64,i64,i64,i64,i64
"""1e9e8ef04dbcff4541ed26657ea517…","""perfumaria""",40,287,1,225,16,10,14
"""3aa071139cb16b67ca9e5dea641aaa…","""artes""",44,276,1,1000,30,18,20


In [94]:
product.select(
    pl.all().null_count()
).transpose(include_header=True)

column,column_0
str,u32
"""product_id""",0
"""product_category_name""",610
"""product_name_lenght""",610
"""product_description_lenght""",610
"""product_photos_qty""",610
"""product_weight_g""",2
"""product_length_cm""",2
"""product_height_cm""",2
"""product_width_cm""",2


In [95]:
product_name = pl.read_csv(
    source = "dataset/product_category_name_translation.csv"
)
product_name.head(2)

product_category_name,product_category_name_english
str,str
"""beleza_saude""","""health_beauty"""
"""informatica_acessorios""","""computers_accessories"""


In [96]:
product_name = product_name.with_columns(
    pl.col("product_category_name")\
        .str.replace_all('_',' ')\
        .str.to_titlecase()
)

In [97]:
product_name.write_csv(
    "dataset_limpo_para_analise/olist_products_name.csv" 
)

#### 

#### limpeza para location(olist_geolocatoin)

In [98]:
geolocation = pl.read_csv(
    source = "dataset/olist_geolocation_dataset.csv",
    infer_schema=False
)

In [99]:
geolocation['geolocation_city'].unique().to_numpy().tolist()

['duere',
 'monte alegre dos campos',
 'simao pereira',
 'igarape do meio',
 'fazenda vilanova',
 'maximiliano de almeida',
 'maetinga',
 'rj',
 'mirandópolis',
 'santa rita',
 'reserva do iguaçu',
 'brazlandia',
 'piedade dos gerais',
 'barra do jacuipe',
 'augustinopolis',
 'acu',
 'luziapolis',
 'caseara',
 'barra dos coqueiros',
 'lagoa do sitio',
 'sao jose de mipibu',
 'estação cocal',
 'pauini',
 'sao joao das duas pontes',
 'nova milano',
 'croata',
 'sao benedito da cachoeirinha',
 'brejo do piaui',
 'roseira',
 'ichu',
 'guapore',
 'cacimbinhas',
 'jussara',
 'poco fundo',
 'vista alegre',
 'candeias',
 'itambacuri',
 'santana do ipanema',
 'brazlândia',
 'jau',
 'matinha',
 'santa rosa de goias',
 'uniflor',
 'miraí',
 'cachoeira',
 'água fria',
 'aliança do tocantins',
 'passa tempo',
 'ibirapitanga',
 'casa nova',
 'vitória das missões',
 'coração de jesus',
 'jeremoabo',
 'nova santa rosa',
 'três barras',
 'zortéa',
 'pimenteiras',
 'luiziania',
 'guzolândia',
 'marituba